# Module 5: Edge Detection

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/05_Module.ipynb)

**Week 9-10: Gradient-Based Methods, Canny Edge Detection**

## Learning Objectives
- Understand edge detection theory and gradient computation
- Apply Sobel, Prewitt, and Scharr operators
- Implement Laplacian of Gaussian (LoG)
- Master Canny edge detection algorithm
- Compare different edge detection methods

---
## ⚠️ IMPORTANT: Run All Cells in Order

**This notebook must be executed sequentially from top to bottom.**

- Click **Runtime → Run all** (or **Cell → Run All**)
- Do NOT skip cells or run them out of order
- Each cell depends on variables from previous cells

---

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
print(f"OpenCV version: {cv2.__version__}")

---
## Mathematical Foundations: Edge Detection Theory

### What is an Edge?

An **edge** is a rapid change in image intensity:
- Discontinuity in intensity
- Boundary between regions
- Corresponds to object boundaries, texture changes, shadows

### Why Edge Detection?

1. **Data reduction**: Edges contain most important image information
2. **Feature extraction**: Basis for object recognition
3. **Image understanding**: Segment and analyze image structure
4. **Preprocessing**: For segmentation, tracking, registration

### Edge Types

1. **Step edge**: Abrupt intensity change
2. **Ramp edge**: Gradual transition
3. **Roof edge**: Peak in intensity
4. **Line edge**: Thin structure

### 1. Image Gradient

The **gradient** of an image $f(x, y)$ is a vector:

$$
\nabla f = \begin{bmatrix} \frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y} \end{bmatrix} = \begin{bmatrix} G_x \\ G_y \end{bmatrix}
$$

#### Gradient Magnitude

The **magnitude** (strength) of the edge:

$$
|\nabla f| = \sqrt{G_x^2 + G_y^2}
$$

**Approximation** (faster computation):

$$
|\nabla f| \approx |G_x| + |G_y|
$$

#### Gradient Direction

The **direction** of the edge (perpendicular to gradient):

$$
\theta = \arctan\left(\frac{G_y}{G_x}\right)
$$

### Discrete Approximation

In discrete images, derivatives are approximated using **finite differences**:

**Forward difference**:
$$
\frac{\partial f}{\partial x} \approx f(x+1, y) - f(x, y)
$$

**Backward difference**:
$$
\frac{\partial f}{\partial x} \approx f(x, y) - f(x-1, y)
$$

**Central difference** (more accurate):
$$
\frac{\partial f}{\partial x} \approx \frac{f(x+1, y) - f(x-1, y)}{2}
$$

In [ ]:
# Load sample image
import urllib.request
from urllib.request import Request, urlopen
import numpy as np

url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Valve_original_%281%29.PNG/300px-Valve_original_%281%29.PNG'

# Download with User-Agent header to avoid 403 Forbidden
try:
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    with urlopen(req) as response, open('valve.png', 'wb') as out_file:
        out_file.write(response.read())
    print("Image downloaded successfully!")
except Exception as e:
    print(f"Download failed: {e}")
    print("Creating placeholder image...")
    # Create a placeholder image if download fails
    placeholder = np.random.randint(0, 256, (300, 300), dtype=np.uint8)
    import cv2
    cv2.imwrite('valve.png', placeholder)

img = cv2.imread('valve.png', cv2.IMREAD_GRAYSCALE)
print(f"Image shape: {img.shape}")
print(f"Dtype: {img.dtype}")

plt.figure(figsize=(8, 8))
plt.imshow(img, cmap='gray')
plt.title('Original Image (Grayscale)')
plt.colorbar()
plt.axis('off')
plt.show()


### 2. Sobel Operator

The **Sobel operator** combines gradient computation with smoothing.

#### Sobel Kernels

**Horizontal gradient** $G_x$ (detects vertical edges):
$$
S_x = \begin{bmatrix}
-1 & 0 & +1 \\
-2 & 0 & +2 \\
-1 & 0 & +1
\end{bmatrix}
$$

**Vertical gradient** $G_y$ (detects horizontal edges):
$$
S_y = \begin{bmatrix}
-1 & -2 & -1 \\
0 & 0 & 0 \\
+1 & +2 & +1
\end{bmatrix}
$$

#### Decomposition

Sobel = Smoothing (perpendicular) + Differentiation

$$
S_x = \begin{bmatrix} 1 \\ 2 \\ 1 \end{bmatrix} \times \begin{bmatrix} -1 & 0 & 1 \end{bmatrix}
$$

- Column vector: Gaussian-like smoothing in $y$ direction
- Row vector: Central difference in $x$ direction

#### Properties

1. **Noise reduction**: Built-in smoothing reduces noise sensitivity
2. **Separability**: Can be computed as two 1D convolutions
3. **Isotropy**: Reasonably uniform response in all directions

In [ ]:
# Sobel edge detection
print("=" * 70)
print("SOBEL EDGE DETECTION")
print("=" * 70)

# Compute gradients
sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)

# Compute magnitude
sobel_mag = np.sqrt(sobel_x**2 + sobel_y**2)
sobel_mag_approx = np.abs(sobel_x) + np.abs(sobel_y)

# Compute direction
sobel_dir = np.arctan2(sobel_y, sobel_x) * 180 / np.pi

print(f"\nGradient X range: [{sobel_x.min():.2f}, {sobel_x.max():.2f}]")
print(f"Gradient Y range: [{sobel_y.min():.2f}, {sobel_y.max():.2f}]")
print(f"Magnitude range: [{sobel_mag.min():.2f}, {sobel_mag.max():.2f}]")
print(f"Direction range: [{sobel_dir.min():.2f}°, {sobel_dir.max():.2f}°]")

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(sobel_x, cmap='RdBu')
axes[0, 1].set_title('Sobel X\n(Vertical Edges)')
axes[0, 1].axis('off')

axes[0, 2].imshow(sobel_y, cmap='RdBu')
axes[0, 2].set_title('Sobel Y\n(Horizontal Edges)')
axes[0, 2].axis('off')

axes[1, 0].imshow(sobel_mag, cmap='hot')
axes[1, 0].set_title('Gradient Magnitude\n(Exact)')
axes[1, 0].axis('off')

axes[1, 1].imshow(sobel_mag_approx, cmap='hot')
axes[1, 1].set_title('Gradient Magnitude\n(Approximation)')
axes[1, 1].axis('off')

axes[1, 2].imshow(sobel_dir, cmap='hsv')
axes[1, 2].set_title('Gradient Direction\n(Angle in degrees)')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Sobel combines smoothing + differentiation!")

### 3. Other Gradient Operators

#### Prewitt Operator

Similar to Sobel but with uniform smoothing:

$$
P_x = \begin{bmatrix}
-1 & 0 & +1 \\
-1 & 0 & +1 \\
-1 & 0 & +1
\end{bmatrix}, \quad
P_y = \begin{bmatrix}
-1 & -1 & -1 \\
0 & 0 & 0 \\
+1 & +1 & +1
\end{bmatrix}
$$

#### Scharr Operator

Improved rotational symmetry (better than Sobel for small kernels):

$$
\text{Scharr}_x = \begin{bmatrix}
-3 & 0 & +3 \\
-10 & 0 & +10 \\
-3 & 0 & +3
\end{bmatrix}, \quad
\text{Scharr}_y = \begin{bmatrix}
-3 & -10 & -3 \\
0 & 0 & 0 \\
+3 & +10 & +3
\end{bmatrix}
$$

#### Roberts Cross Operator

Simplest 2×2 gradient operator:

$$
R_x = \begin{bmatrix}
+1 & 0 \\
0 & -1
\end{bmatrix}, \quad
R_y = \begin{bmatrix}
0 & +1 \\
-1 & 0
\end{bmatrix}
$$

In [ ]:
# Compare different gradient operators
print("=" * 70)
print("GRADIENT OPERATORS COMPARISON")
print("=" * 70)

# Prewitt
prewitt_x = cv2.filter2D(img, cv2.CV_64F, 
                         np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]))
prewitt_y = cv2.filter2D(img, cv2.CV_64F,
                         np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]))
prewitt_mag = np.sqrt(prewitt_x**2 + prewitt_y**2)

# Sobel (already computed)
# sobel_mag (from previous cell)

# Scharr
scharr_x = cv2.Scharr(img, cv2.CV_64F, 1, 0)
scharr_y = cv2.Scharr(img, cv2.CV_64F, 0, 1)
scharr_mag = np.sqrt(scharr_x**2 + scharr_y**2)

# Roberts
roberts_x = cv2.filter2D(img, cv2.CV_64F,
                         np.array([[1, 0], [0, -1]]))
roberts_y = cv2.filter2D(img, cv2.CV_64F,
                         np.array([[0, 1], [-1, 0]]))
roberts_mag = np.sqrt(roberts_x**2 + roberts_y**2)

print(f"\nOperator Kernels:")
print(f"  Prewitt:  3×3, uniform smoothing")
print(f"  Sobel:    3×3, Gaussian-like smoothing (weights: 1-2-1)")
print(f"  Scharr:   3×3, optimized rotational symmetry (weights: 3-10-3)")
print(f"  Roberts:  2×2, minimal smoothing, high noise sensitivity")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(prewitt_mag, cmap='hot')
axes[0, 0].set_title('Prewitt', fontsize=14)
axes[0, 0].axis('off')

axes[0, 1].imshow(sobel_mag, cmap='hot')
axes[0, 1].set_title('Sobel', fontsize=14)
axes[0, 1].axis('off')

axes[1, 0].imshow(scharr_mag, cmap='hot')
axes[1, 0].set_title('Scharr', fontsize=14)
axes[1, 0].axis('off')

axes[1, 1].imshow(roberts_mag, cmap='hot')
axes[1, 1].set_title('Roberts Cross', fontsize=14)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Scharr > Sobel > Prewitt in accuracy; Roberts is fastest but noisiest")

### 4. Laplacian Operator

The **Laplacian** is a **second derivative** operator:

$$
\nabla^2 f = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}
$$

#### Discrete Laplacian Kernel

**Standard 4-neighbor**:
$$
L_4 = \begin{bmatrix}
0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0
\end{bmatrix}
$$

**8-neighbor** (includes diagonals):
$$
L_8 = \begin{bmatrix}
1 & 1 & 1 \\
1 & -8 & 1 \\
1 & 1 & 1
\end{bmatrix}
$$

#### Properties

1. **Zero-crossing**: Edges occur where Laplacian crosses zero
2. **Isotropic**: Rotation invariant
3. **Noise sensitive**: Second derivative amplifies noise!
4. **No direction**: Only magnitude, no edge orientation

### Laplacian of Gaussian (LoG)

**Problem**: Laplacian is very sensitive to noise.

**Solution**: Smooth with Gaussian first!

$$
\text{LoG}(x, y) = \nabla^2 [G(x, y, \sigma) * f(x, y)]
$$

Using calculus, this equals:

$$
\text{LoG}(x, y) = [\nabla^2 G(x, y, \sigma)] * f(x, y)
$$

The **LoG kernel** ("Mexican hat" function):

$$
\text{LoG}(x, y) = -\frac{1}{\pi\sigma^4}\left[1 - \frac{x^2 + y^2}{2\sigma^2}\right]e^{-\frac{x^2 + y^2}{2\sigma^2}}
$$

- $\sigma$ controls scale (amount of smoothing)
- Larger $\sigma$ → detects coarser edges

In [ ]:
# Laplacian edge detection
print("=" * 70)
print("LAPLACIAN AND LOG EDGE DETECTION")
print("=" * 70)

# Direct Laplacian (very noisy!)
laplacian_direct = cv2.Laplacian(img, cv2.CV_64F, ksize=3)

# LoG: Gaussian blur first, then Laplacian
sigma = 2.0
img_blur = cv2.GaussianBlur(img, (0, 0), sigma)
laplacian_log = cv2.Laplacian(img_blur, cv2.CV_64F, ksize=3)

# Alternative: Use cv2.Laplacian with larger kernel
laplacian_5x5 = cv2.Laplacian(img, cv2.CV_64F, ksize=5)

# Create LoG kernel manually
def create_log_kernel(sigma, size=None):
    if size is None:
        size = int(8 * sigma + 1)
        if size % 2 == 0:
            size += 1
    
    ax = np.arange(-size // 2 + 1., size // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    
    kernel = -1 / (np.pi * sigma**4) * (1 - (xx**2 + yy**2) / (2 * sigma**2)) * \
             np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    
    return kernel

log_kernel = create_log_kernel(sigma=2)
laplacian_log_custom = cv2.filter2D(img, cv2.CV_64F, log_kernel)

print(f"\nLoG kernel size: {log_kernel.shape}")
print(f"Sigma: {sigma}")
print(f"\nLaplacian (direct) range: [{laplacian_direct.min():.2f}, {laplacian_direct.max():.2f}]")
print(f"LoG range: [{laplacian_log.min():.2f}, {laplacian_log.max():.2f}]")

# Visualize LoG kernel
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 3D plot of LoG kernel
from mpl_toolkits.mplot3d import Axes3D
ax = fig.add_subplot(131, projection='3d')
x = np.arange(log_kernel.shape[0])
y = np.arange(log_kernel.shape[1])
X, Y = np.meshgrid(x, y)
ax.plot_surface(X, Y, log_kernel, cmap='coolwarm')
ax.set_title('LoG Kernel (3D)\n"Mexican Hat"')

# 2D heatmap
axes[1].imshow(log_kernel, cmap='RdBu')
axes[1].set_title('LoG Kernel (2D)')
axes[1].colorbar()

# Cross-section
mid = log_kernel.shape[0] // 2
axes[2].plot(log_kernel[mid, :], 'b-', linewidth=2, label='Horizontal')
axes[2].plot(log_kernel[:, mid], 'r--', linewidth=2, label='Vertical')
axes[2].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[2].grid(True, alpha=0.3)
axes[2].set_title('LoG Cross-Section')
axes[2].legend()

plt.tight_layout()
plt.show()

# Compare results
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('Original')
axes[0, 0].axis('off')

axes[0, 1].imshow(laplacian_direct, cmap='RdBu')
axes[0, 1].set_title('Laplacian (Direct)\nVery Noisy!')
axes[0, 1].axis('off')

axes[1, 0].imshow(laplacian_log, cmap='RdBu')
axes[1, 0].set_title(f'LoG (Gaussian σ={sigma})\nMuch Cleaner!')
axes[1, 0].axis('off')

axes[1, 1].imshow(laplacian_log_custom, cmap='RdBu')
axes[1, 1].set_title('LoG (Custom Kernel)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Always use LoG instead of raw Laplacian!")

### 5. Canny Edge Detector

The **Canny edge detector** is the most popular edge detection algorithm.

#### Design Criteria (Canny, 1986)

1. **Good detection**: Low error rate (detect real edges, minimize false positives)
2. **Good localization**: Edge points close to true edge centers
3. **Single response**: One detector response per edge

#### Algorithm Steps

**Step 1: Noise Reduction**

Apply Gaussian filter:
$$
I_\text{smooth} = G(\sigma) * I
$$

**Step 2: Gradient Calculation**

Compute magnitude and direction:
$$
G = \sqrt{G_x^2 + G_y^2}, \quad \theta = \arctan\left(\frac{G_y}{G_x}\right)
$$

**Step 3: Non-Maximum Suppression (NMS)**

Thin edges to 1-pixel width:
- For each pixel, check if it's a local maximum along gradient direction
- Suppress if neighbors in gradient direction have higher magnitude

**Step 4: Double Threshold**

Classify edges:
- $G > T_\text{high}$: **Strong edge**
- $T_\text{low} < G < T_\text{high}$: **Weak edge**
- $G < T_\text{low}$: **Non-edge** (suppress)

**Step 5: Edge Tracking by Hysteresis**

Connect edges:
- Keep all strong edges
- Keep weak edges **only if** connected to strong edges
- Suppress isolated weak edges

#### Parameters

- $\sigma$: Gaussian smoothing (typical: 1-2)
- $T_\text{low}$: Low threshold (typical: 50-100)
- $T_\text{high}$: High threshold (typical: 100-200)
- Ratio $T_\text{high} / T_\text{low}$: typically 2:1 or 3:1

In [ ]:
# Canny edge detection
print("=" * 70)
print("CANNY EDGE DETECTION")
print("=" * 70)

# Apply Canny with different thresholds
canny_strict = cv2.Canny(img, 150, 200)  # High thresholds
canny_medium = cv2.Canny(img, 100, 150)  # Medium thresholds
canny_loose = cv2.Canny(img, 50, 100)    # Low thresholds

print(f"\nCanny parameters:")
print(f"  Strict:  T_low=150, T_high=200 (ratio 1.33)")
print(f"  Medium:  T_low=100, T_high=150 (ratio 1.50)")
print(f"  Loose:   T_low=50,  T_high=100 (ratio 2.00)")

# Count edge pixels
print(f"\nEdge pixel counts:")
print(f"  Strict:  {np.sum(canny_strict > 0)} pixels")
print(f"  Medium:  {np.sum(canny_medium > 0)} pixels")
print(f"  Loose:   {np.sum(canny_loose > 0)} pixels")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(canny_strict, cmap='gray')
axes[0, 1].set_title('Canny (Strict)\nT_low=150, T_high=200')
axes[0, 1].axis('off')

axes[1, 0].imshow(canny_medium, cmap='gray')
axes[1, 0].set_title('Canny (Medium)\nT_low=100, T_high=150')
axes[1, 0].axis('off')

axes[1, 1].imshow(canny_loose, cmap='gray')
axes[1, 1].set_title('Canny (Loose)\nT_low=50, T_high=100')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Lower thresholds detect more edges but include more noise!")

In [ ]:
# Demonstrate Canny algorithm steps
print("=" * 70)
print("CANNY ALGORITHM: STEP-BY-STEP BREAKDOWN")
print("=" * 70)

# Step 1: Gaussian smoothing
sigma = 1.4
img_smooth = cv2.GaussianBlur(img, (5, 5), sigma)
print(f"\nStep 1: Gaussian smoothing (σ={sigma})")

# Step 2: Gradient calculation
grad_x = cv2.Sobel(img_smooth, cv2.CV_64F, 1, 0, ksize=3)
grad_y = cv2.Sobel(img_smooth, cv2.CV_64F, 0, 1, ksize=3)
magnitude = np.sqrt(grad_x**2 + grad_y**2)
direction = np.arctan2(grad_y, grad_x)
print(f"Step 2: Gradient computation (Sobel)")
print(f"  Magnitude range: [{magnitude.min():.2f}, {magnitude.max():.2f}]")

# Step 3: Non-maximum suppression (simplified version)
def non_max_suppression_simple(mag, angle):
    """Simplified NMS for demonstration"""
    M, N = mag.shape
    nms = np.zeros((M, N), dtype=np.float64)
    angle = angle * 180. / np.pi
    angle[angle < 0] += 180
    
    for i in range(1, M-1):
        for j in range(1, N-1):
            q = 255
            r = 255
            
            # Angle 0
            if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                q = mag[i, j+1]
                r = mag[i, j-1]
            # Angle 45
            elif (22.5 <= angle[i,j] < 67.5):
                q = mag[i+1, j-1]
                r = mag[i-1, j+1]
            # Angle 90
            elif (67.5 <= angle[i,j] < 112.5):
                q = mag[i+1, j]
                r = mag[i-1, j]
            # Angle 135
            elif (112.5 <= angle[i,j] < 157.5):
                q = mag[i-1, j-1]
                r = mag[i+1, j+1]
            
            if (mag[i,j] >= q) and (mag[i,j] >= r):
                nms[i,j] = mag[i,j]
            else:
                nms[i,j] = 0
    
    return nms

nms_result = non_max_suppression_simple(magnitude, direction)
print(f"Step 3: Non-maximum suppression")
print(f"  Thinned edges to 1-pixel width")

# Step 4 & 5: Double thresholding and hysteresis
low_thresh = 50
high_thresh = 150
strong_edges = (nms_result >= high_thresh).astype(np.uint8) * 255
weak_edges = ((nms_result >= low_thresh) & (nms_result < high_thresh)).astype(np.uint8) * 128
print(f"Step 4: Double thresholding (T_low={low_thresh}, T_high={high_thresh})")

# Full Canny result
canny_result = cv2.Canny(img, low_thresh, high_thresh)
print(f"Step 5: Hysteresis (connect weak edges to strong edges)")

# Visualize all steps
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('0. Original')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_smooth, cmap='gray')
axes[0, 1].set_title('1. Gaussian Blur')
axes[0, 1].axis('off')

axes[0, 2].imshow(magnitude, cmap='hot')
axes[0, 2].set_title('2. Gradient Magnitude')
axes[0, 2].axis('off')

axes[0, 3].imshow(direction, cmap='hsv')
axes[0, 3].set_title('2. Gradient Direction')
axes[0, 3].axis('off')

axes[1, 0].imshow(nms_result, cmap='gray')
axes[1, 0].set_title('3. Non-Max Suppression')
axes[1, 0].axis('off')

axes[1, 1].imshow(strong_edges + weak_edges, cmap='gray')
axes[1, 1].set_title('4. Double Threshold\n(White=Strong, Gray=Weak)')
axes[1, 1].axis('off')

axes[1, 2].imshow(canny_result, cmap='gray')
axes[1, 2].set_title('5. Final Canny Edges\n(After Hysteresis)')
axes[1, 2].axis('off')

axes[1, 3].imshow(img, cmap='gray')
axes[1, 3].imshow(canny_result, cmap='Reds', alpha=0.5)
axes[1, 3].set_title('Overlay on Original')
axes[1, 3].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: Canny combines multiple techniques for optimal edge detection!")

## 5.7 Comparison of Edge Detection Methods

### Summary Table

| Method | Type | Noise Sensitivity | Edge Localization | Complexity | Best For |
|--------|------|-------------------|-------------------|------------|----------|
| **Roberts** | 1st derivative | Very High | Good | Very Low | Clean images, speed critical |
| **Prewitt** | 1st derivative | High | Good | Low | Simple applications |
| **Sobel** | 1st derivative | Medium | Good | Low | General purpose |
| **Scharr** | 1st derivative | Medium | Better | Low | Better accuracy than Sobel |
| **Laplacian** | 2nd derivative | Very High | Excellent | Low | Not recommended (too noisy) |
| **LoG** | 2nd derivative | Low | Excellent | Medium | Blob detection, zero-crossings |
| **Canny** | Multi-stage | Very Low | Excellent | High | Best overall edge detection |

### When to Use Each Method

**Sobel/Scharr**: 
- Fast, simple edge detection
- When you need gradient direction
- Preprocessing for other algorithms

**LoG**:
- Scale-space analysis
- Blob detection
- When you need zero-crossings

**Canny**:
- Best general-purpose edge detector
- When edge quality is critical
- Object detection, segmentation
- When you can afford higher computation cost

In [ ]:
# Final comprehensive comparison
print("=" * 70)
print("FINAL COMPARISON: ALL EDGE DETECTION METHODS")
print("=" * 70)

# Load a test image with some noise
test_img = img.copy()
# Add Gaussian noise
noise = np.random.normal(0, 10, test_img.shape)
test_img_noisy = np.clip(test_img + noise, 0, 255).astype(np.uint8)

# Apply all methods
sobel_x = cv2.Sobel(test_img_noisy, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(test_img_noisy, cv2.CV_64F, 0, 1, ksize=3)
edges_sobel = np.sqrt(sobel_x**2 + sobel_y**2)

scharr_x = cv2.Scharr(test_img_noisy, cv2.CV_64F, 1, 0)
scharr_y = cv2.Scharr(test_img_noisy, cv2.CV_64F, 0, 1)
edges_scharr = np.sqrt(scharr_x**2 + scharr_y**2)

edges_laplacian = cv2.Laplacian(test_img_noisy, cv2.CV_64F)
edges_laplacian = np.abs(edges_laplacian)

img_blur = cv2.GaussianBlur(test_img_noisy, (5, 5), 2)
edges_log = cv2.Laplacian(img_blur, cv2.CV_64F)
edges_log = np.abs(edges_log)

edges_canny = cv2.Canny(test_img_noisy, 50, 150)

# Visualize comparison
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(test_img_noisy, cmap='gray')
axes[0, 0].set_title('Input (with noise)', fontsize=12)
axes[0, 0].axis('off')

axes[0, 1].imshow(edges_sobel, cmap='hot')
axes[0, 1].set_title('Sobel\n(1st derivative)', fontsize=12)
axes[0, 1].axis('off')

axes[0, 2].imshow(edges_scharr, cmap='hot')
axes[0, 2].set_title('Scharr\n(Better 1st derivative)', fontsize=12)
axes[0, 2].axis('off')

axes[1, 0].imshow(edges_laplacian, cmap='hot')
axes[1, 0].set_title('Laplacian\n(2nd derivative, NOISY!)', fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(edges_log, cmap='hot')
axes[1, 1].set_title('LoG\n(Laplacian of Gaussian)', fontsize=12)
axes[1, 1].axis('off')

axes[1, 2].imshow(edges_canny, cmap='gray')
axes[1, 2].set_title('Canny\n(BEST overall!)', fontsize=12)
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("RECOMMENDATION: Use Canny for most applications!")
print("=" * 70)
print("Why Canny is superior:")
print("  1. Optimal edge detection (low error rate)")
print("  2. Good localization (edges at correct positions)")
print("  3. Single response (thin, connected edges)")
print("  4. Low noise sensitivity (built-in smoothing)")
print("  5. Hysteresis prevents broken edges")
print("\nWhen NOT to use Canny:")
print("  - Need gradient direction (use Sobel/Scharr)")
print("  - Need scale-space analysis (use LoG)")
print("  - Extreme speed requirements (use Sobel)")

## Summary

### Key Concepts

1. **Image Gradient**: Vector of partial derivatives $\nabla f = [G_x, G_y]^T$
2. **First Derivatives**: Sobel, Prewitt, Scharr operators
3. **Second Derivatives**: Laplacian, LoG (Laplacian of Gaussian)
4. **Canny Algorithm**: 5-step optimal edge detection
5. **Trade-offs**: Noise vs. detail, speed vs. quality

### Mathematical Formulas Reference

**Gradient**:
$$
\nabla f = \begin{bmatrix} \partial f/\partial x \\ \partial f/\partial y \end{bmatrix}, \quad
|\nabla f| = \sqrt{G_x^2 + G_y^2}, \quad
\theta = \arctan(G_y/G_x)
$$

**Laplacian**:
$$
\nabla^2 f = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}
$$

**LoG**:
$$
\text{LoG}(x,y) = -\frac{1}{\pi\sigma^4}\left[1 - \frac{x^2+y^2}{2\sigma^2}\right]e^{-\frac{x^2+y^2}{2\sigma^2}}
$$

### OpenCV Functions Reference

| Operation | Function | Key Parameters |
|-----------|----------|----------------|
| Sobel | `cv2.Sobel(img, ddepth, dx, dy, ksize)` | dx, dy: derivative order |
| Scharr | `cv2.Scharr(img, ddepth, dx, dy)` | More accurate than Sobel |
| Laplacian | `cv2.Laplacian(img, ddepth, ksize)` | Second derivative |
| Canny | `cv2.Canny(img, threshold1, threshold2)` | threshold1/2: hysteresis |

**Next**: Module 6 - Feature Detection